In [1]:
import os
import sys
sys.path.append('/mnt/c/Users/44yos/RacePrediction/resnet_win5')
sys.path.append('/usr/local/bin/chromedriver')
from data_setup.race_data.this_week_race_url import this_week_race_url
from data_setup.race_data.this_week_race_html import this_week_race_html
from data_setup.race_data.make_csv_from_this_week_html import make_csv_from_this_week_html
from data_setup.race_data.make_csv_from_this_week_html_selenium import make_csv_from_this_week_html_selenium
# from data_setup.race_data.make_csv_this_week_with_odds import make_csv_this_week_with_odds
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os
from os import path

from pickle import load

from models import resnet
from tensorflow.python import keras
from tensorflow.python.keras.models import load_model
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

In [2]:
this_week_race_url("12_25")

In [3]:
# this_week_race_html()

In [4]:
os.makedirs('../data/csv/2022/Dec_5', exist_ok = True)
make_csv_from_this_week_html_selenium("Dec_5", "12_25", "2022-12-25")

In [5]:
#make_csv_this_week_with_odds()

In [6]:
race_df = pd.read_csv("../data/csv/2022/Dec_5/12_25.csv", sep = ",")

In [7]:
# print(len(race_df['race_id'].unique()))
# print(race_df.shape)
# print(race_df.head(2))

In [8]:
# odds = pd.DataFrame(columns = ["odds"])
# odds["odds"] = [3.8,36.2,389.8,11.1,61.0,36.2,19.0,29.7,13.0,52.7,8.3,3.0,82.9,4.6,40.0,25.2,3.7,7.7,2.0,70.6,118.3,19.1,22.7,186.0,194.0,86.7,159.9,18.5,77.8,8.2,8.7,70.7,37.0,17.9,22.1,57.4,6.2,8.2,135.9,47.5,2.2,70.4,21.2,114.6,5.1,27.0,37.7,8.3,57.7,94.5,11.8,53.2,7.1,8.7,9.4,115.8,2.9,17.0,134.3,89.3,3.5,121.7,68.7,30.4,21.4,17.0,8.1,11.2,55.8,3.8,38.3,31.6,2.8,24.3,12.8,10.9,13.0,60.5,22.1,2.5,32.0,5.9,43.1,37.7,24.6,80.0,25.5,11.2,35.0,63.1,9.1,16.5,4.8,10.0,5.6,2.8,33.1,18.4,4.6,27.0,7.9,60.8,13.8,42.9,4.9,79.4,33.6,8.0,4.2,42.7,33.4,67.2,38.7,2.3,60.5,7.6,14.9,2.4,71.0,38.1,119.2,9.4,20.4]

In [9]:
# print(odds.shape)
# print(odds)

In [10]:
# race_df["odds"] = odds["odds"]

In [11]:
# race_df.tail(50)

race_round

In [12]:
race_df['race_round'] = race_df['race_round'].str.strip('R \n')

In [13]:
race_df.head(5)

,race_id,race_round,race_title,race_distance,weather,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight
0,202206050808&rf,8,冬至特別,ダ1200m (右),晴,良,13:25発走,2022-12-25,中山,16,?,1,1,2019104518,牝3,54.0,1170,?,?,?,3.6,?
1,202206050808&rf,8,冬至特別,ダ1200m (右),晴,良,13:25発走,2022-12-25,中山,16,?,1,2,2016110159,牡6,57.0,1092,?,?,?,23.0,?
2,202206050808&rf,8,冬至特別,ダ1200m (右),晴,良,13:25発走,2022-12-25,中山,16,?,2,3,2016101493,牡6,57.0,1192,?,?,?,168.0,?
3,202206050808&rf,8,冬至特別,ダ1200m (右),晴,良,13:25発走,2022-12-25,中山,16,?,2,4,2018101817,牡4,57.0,732,?,?,?,17.7,?
4,202206050808&rf,8,冬至特別,ダ1200m (右),晴,良,13:25発走,2022-12-25,中山,16,?,3,5,2017102984,牝5,55.0,1146,?,?,?,90.8,?


race_rank

In [14]:
race_df["race_title"] = race_df["race_title"].replace('有馬記念','有馬記念G1')
# race_df["race_title"] = race_df["race_title"].replace('キーンランドC','キーンランドCG3')
# race_df["race_title"] = race_df["race_title"].replace('新潟2歳S','新潟2歳SG3')
# race_df["race_title"] = race_df["race_title"].replace('府中牝馬S','府中牝馬SG2')
# race_df["race_title"] = race_df["race_title"].replace('神戸新聞杯','神戸新聞杯G2')

In [15]:
race_rank = race_df["race_title"].str.extract('(G1|G2|G3)', expand = True)
race_rank.columns = {"race_rank"}

race_df = pd.concat([race_df, race_rank], axis = 1)
race_df.fillna(value = {'race_rank': 0}, inplace = True) # 元もデータに反映
race_df['race_rank'] = race_df['race_rank'].replace('G1', 3)
race_df['race_rank'] = race_df['race_rank'].replace('G2', 2)
race_df['race_rank'] = race_df['race_rank'].replace('G3', 1)

In [16]:
print("race_rank:", race_df["race_rank"].value_counts())

race_rank: 0    107
3     16
Name: race_rank, dtype: int64


In [17]:
# race_df.head(5)

race_distance

In [18]:
# race_df["race_distance"].unique()

In [19]:
ground_type = race_df["race_distance"].str.extract('(芝|ダ)', expand = True)
circle = race_df["race_distance"].str.extract('(右|左)', expand = True)
distance = race_df["race_distance"].str.extract('(\d+)m', expand = True)

ground_type.columns = {"ground_type"}
circle.columns = {"circle"}
distance.columns = {"distance"}

race_df = pd.concat([race_df, ground_type], axis = 1)
race_df = pd.concat([race_df, circle], axis = 1)
race_df = pd.concat([race_df, distance], axis = 1)

race_df = pd.get_dummies(race_df, columns = ["ground_type", "circle"])

In [20]:
race_df["distance"] = race_df["distance"].astype(int)
race_df.drop(['race_distance'], axis = 1, inplace = True)

In [21]:
# # 存在しない右左回り適宜追加 ######### circle_左 or circle_右
add_circle = pd.DataFrame(columns = ["circle_右","circle_左"])
race_df = pd.concat([race_df,add_circle], axis = 1).fillna(0)

In [22]:
# race_df.head(5)

weather

In [23]:
# race_df["weather"] = race_df["weather"].mask(race_df["place"] == '小倉', '晴')
# race_df["weather"] = race_df["weather"].mask(race_df["place"] == '阪神', '曇')
# race_df["weather"] = race_df["weather"].mask(race_df["place"] == '中山', '曇')
# race_df["weather"] = race_df["weather"].mask(race_df["place"] == '札幌', '晴')

In [24]:
weather = race_df["weather"].str.extract('(晴|曇|小雨|雨|小雪|雪)', expand = True)
weather.columns = {"weather_circumstance"}
race_df = pd.concat([race_df, weather], axis = 1)
race_df = pd.get_dummies(race_df, columns = ["weather_circumstance"])
race_df.drop(['weather'], axis = 1, inplace = True)

In [25]:
# race_df.head(5)

In [26]:
# 存在しない天気適宜追加 #########
add_weather = pd.DataFrame(columns = ["weather_circumstance_小雨","weather_circumstance_雨","weather_circumstance_晴","weather_circumstance_曇","weather_circumstance_小雪","weather_circumstance_雪"])
race_df = pd.concat([race_df,add_weather], axis = 1).fillna(0)

In [27]:
# race_df.head(5)

ground_condition

In [28]:
# race_df["ground_condition"] = race_df["ground_condition"].mask(race_df["place"] == '小倉', '良')
# race_df["ground_condition"] = race_df["ground_condition"].mask(race_df["place"] == '中山', '稍重')
# race_df["ground_condition"] = race_df["ground_condition"].mask(race_df["place"] == '中京', '稍重')
# race_df["ground_condition"] = "良"

In [29]:
race_df["ground_condition"] = race_df["ground_condition"].replace('(不良)', 4, regex = True)
race_df["ground_condition"] = race_df["ground_condition"].replace('(稍重)', 2, regex = True)
race_df["ground_condition"] = race_df["ground_condition"].replace('(重)', 3, regex = True)
race_df["ground_condition"] = race_df["ground_condition"].replace('(良)', 1, regex = True)

In [30]:
print("ground_condition:", race_df["ground_condition"].value_counts())

ground_condition: 1    123
Name: ground_condition, dtype: int64


In [31]:
# race_df.head(5)

datetime

In [32]:
race_df["date"] =  pd.to_datetime(race_df['date'])
race_df.drop(["time"], axis = 1, inplace = True)

In [33]:
# race_df.head(5)

place

In [34]:
race_df = pd.get_dummies(race_df, columns = ["place"])

In [35]:
# race_df.head(5)

In [36]:
# 存在しない場所追加 '札幌', '中京', '新潟', '函館', '小倉', '京都','東京','福島', '阪神' '中山'
add_place = pd.DataFrame(columns =['札幌','中京','新潟','函館','小倉','京都','東京','福島'])
race_df = pd.concat([race_df,add_place], axis = 1).fillna(0)

In [37]:
# race_df.head(5)

sex_age

In [38]:
sex = race_df["sex_and_age"].str.extract('(牡|牝|セ)', expand = True)
sex.columns = {"sex"}
race_df = pd.concat([race_df, sex], axis = 1)
race_df = pd.get_dummies(race_df, columns = ["sex"])

race_df["sex_and_age"] = race_df["sex_and_age"].str.strip("牡牝セ\n")
race_df["sex_and_age"] = race_df["sex_and_age"].astype(int)
race_df = race_df.rename(columns = {"sex_and_age": "age"})

In [39]:
# race_df.head(5)

burden_weight

In [40]:
race_df = race_df[race_df['burden_weight'] != '未定']

In [41]:
race_df["burden_weight"] = race_df["burden_weight"].astype(float)

odds

In [42]:
# print(race_df["odds"].dtypes)

In [43]:
# race_df.to_csv("../data/csv/2022/Dec_4/refined_12_24_race.csv", index = False)

In [44]:
#race_df = pd.read_csv("../data/csv/2021/dec_4/refined_12_26_race.csv", sep = ",")

make_train_data

In [45]:
horse_df = pd.read_csv("../data/csv/horse_data/refined_horse_data.csv", sep = ",")
horse_f_df = pd.read_csv("../data/csv/horse_data/horse_featured_data.csv", sep = ",")

/home/yoshi/miniconda3/envs/tensorflow-gpu/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [46]:
race_df['order'] = race_df['order'].astype(str)
race_df['horse_id'] = race_df['horse_id'].astype(str)
race_df['jockey_id'] = race_df['jockey_id'].astype(str)
race_df['date'] = pd.to_datetime(race_df['date'])
# print(race_df.shape)
# print(race_df.dtypes)
# race_df.head(3)

In [47]:
horse_df['order'] = horse_df['order'].astype(str)
horse_df['horse_id'] = horse_df['horse_id'].astype(str)
horse_df['date'] = pd.to_datetime(horse_df['date'])
# print(horse_df.dtypes)
# print(horse_df.shape)
# horse_df.head(3)

In [48]:
horse_f_df['horse_id'] = horse_f_df['horse_id'].astype(str)
# print(horse_f_df.dtypes)
# print(horse_f_df.shape)
# horse_f_df.head(3)

In [49]:
pedigree_map = pd.read_csv("../data/csv/horse_data/pedigree_map.csv", sep = ",")
pedigree_map['horse_id'] = pedigree_map['horse_id'].astype(str)
# print(pedigree_map.dtypes)

add_pedigree_map to race_df

In [50]:
race_df = pd.merge(race_df, pedigree_map, on = "horse_id", how = "left").fillna(0)

In [51]:
# race_df.head(5)

add previous race

In [52]:
#delete unnecessary columns in horse_df
horse_df.drop(["father_id", "grandfather_id", "mother_id", "total_race_number"], axis = 1, inplace = True)

In [53]:
tmp_date_race_data = race_df[["date", "horse_id", "jockey_id"]]

In [54]:
#function 
def make_one_horse_data(tmp_date_horse_data, one_horse_df):
    one_horse_df = one_horse_df.sort_values('date', ascending = False) #False = 近いやつが上
    one_horse_df["same_jockey"] = (tmp_date_horse_data["jockey_id"] == one_horse_df["jockey_id"]) * 1.0
   
    
    one_horse_df = one_horse_df[(one_horse_df["date"] < tmp_date_horse_data["date"])] #レース前を抽出
    
    
    if (len(one_horse_df) <= 2):
        n = len(one_horse_df) 
    else:
        n = 3
    
    #print("one_horse_df")
    #print(one_horse_df)
    
    result_df = one_horse_df.copy()
    result_df.columns = result_df.columns + "_1"
    #print("result_df")
    #print(result_df)
    for i in range(1,n):
        shift_df = one_horse_df.shift(-i)
        #print("shift_df")
        #print(shift_df)
        shift_df.drop(["date"], axis = 1, inplace = True)
        shift_df.drop(["jockey_id"], axis = 1, inplace = True)
        shift_df.columns = shift_df.columns + "_" + str(i+1)
        result_df = pd.concat([result_df, shift_df], axis = 1)
        #print("result_df")
        #print(result_df)
    
    
    return result_df.head(1)                  #result_df.dropna()
        

In [55]:
one_data = tmp_date_race_data.iloc[0].copy()
one_horse_data = horse_df[horse_df["horse_id"] == "2018106606"].copy()
#print(one_data)
#print(one_horse_data)
tmp_race_df = make_one_horse_data(one_data, one_horse_data)
#print(tmp_race_df)

In [56]:
# うまくいくけど時間かかるやつ

for index, tmp in race_df.iterrows():
    tmp_race_df = tmp_race_df.append(make_one_horse_data(tmp, horse_df[horse_df["horse_id"] == tmp["horse_id"]]))


In [57]:
# print(tmp_race_df.head(10))

In [58]:
# tmp_race_df.to_pickle("../data/csv/2021/tmp_race_12_26_df.pkl")
#tmp_race_df = pd.read_pickle("../data/csv/2021/tmp_race_12_25_df.pkl")

In [59]:
#delete horse1,2,3_id
tmp_race_df.drop(['horse_id_2','horse_id_3'], axis = 1, inplace = True)

In [60]:
#tmp_race_df = tmp_race_df.dropna()
tmp_race_df["whole_horse_number_2"] = tmp_race_df["whole_horse_number_2"].fillna(tmp_race_df["whole_horse_number_1"])
tmp_race_df["odds_2"] = tmp_race_df["odds_2"].fillna(tmp_race_df["odds_1"])
tmp_race_df["order_2"] = tmp_race_df["order_2"].fillna(tmp_race_df["order_1"])
tmp_race_df["burden_weight_2"] = tmp_race_df["burden_weight_2"].fillna(tmp_race_df["burden_weight_1"])
tmp_race_df["race_distance_2"] = tmp_race_df["race_distance_2"].fillna(tmp_race_df["race_distance_1"])
tmp_race_df["ground_condition_2"] = tmp_race_df["ground_condition_2"].fillna(tmp_race_df["ground_condition_1"])
tmp_race_df["order_2"] = tmp_race_df["order_2"].fillna(tmp_race_df["order_1"])
tmp_race_df["goal_time_2"] = tmp_race_df["goal_time_2"].fillna(tmp_race_df["goal_time_1"])
#tmp_race_df["velocity_2"] = tmp_race_df["velocity_2"].fillna(tmp_race_df["velocity_1"])
tmp_race_df["half_order_2"] = tmp_race_df["half_order_2"].fillna(tmp_race_df["half_order_1"])
tmp_race_df["last_time_2"] = tmp_race_df["last_time_2"].fillna(tmp_race_df["last_time_1"])
tmp_race_df["horse_weight_2"] = tmp_race_df["horse_weight_2"].fillna(tmp_race_df["horse_weight_1"])
tmp_race_df["weather_circumstance_小雨_2"] = tmp_race_df["weather_circumstance_小雨_2"].fillna(tmp_race_df["weather_circumstance_小雨_1"])
tmp_race_df["weather_circumstance_小雪_2"] = tmp_race_df["weather_circumstance_小雪_2"].fillna(tmp_race_df["weather_circumstance_小雪_1"])
tmp_race_df["weather_circumstance_晴_2"] = tmp_race_df["weather_circumstance_晴_2"].fillna(tmp_race_df["weather_circumstance_晴_1"])
tmp_race_df["weather_circumstance_曇_2"] = tmp_race_df["weather_circumstance_曇_2"].fillna(tmp_race_df["weather_circumstance_曇_1"])
tmp_race_df["weather_circumstance_雨_2"] = tmp_race_df["weather_circumstance_雨_2"].fillna(tmp_race_df["weather_circumstance_雨_1"])
tmp_race_df["weather_circumstance_雪_2"] = tmp_race_df["weather_circumstance_雪_2"].fillna(tmp_race_df["weather_circumstance_雪_1"])
tmp_race_df["main_place_その他_2"] = tmp_race_df["main_place_その他_2"].fillna(tmp_race_df["main_place_その他_1"])
tmp_race_df["main_place_中京_2"] = tmp_race_df["main_place_中京_2"].fillna(tmp_race_df["main_place_中京_1"])
tmp_race_df["main_place_中山_2"] = tmp_race_df["main_place_中山_2"].fillna(tmp_race_df["main_place_中山_1"])
tmp_race_df["main_place_京都_2"] = tmp_race_df["main_place_京都_2"].fillna(tmp_race_df["main_place_京都_1"])
tmp_race_df["main_place_函館_2"] = tmp_race_df["main_place_函館_2"].fillna(tmp_race_df["main_place_函館_1"])
tmp_race_df["main_place_小倉_2"] = tmp_race_df["main_place_小倉_2"].fillna(tmp_race_df["main_place_小倉_1"])
tmp_race_df["main_place_新潟_2"] = tmp_race_df["main_place_新潟_2"].fillna(tmp_race_df["main_place_新潟_1"])
tmp_race_df["main_place_札幌_2"] = tmp_race_df["main_place_札幌_2"].fillna(tmp_race_df["main_place_札幌_1"])
tmp_race_df["main_place_東京_2"] = tmp_race_df["main_place_東京_2"].fillna(tmp_race_df["main_place_東京_1"])
tmp_race_df["main_place_福島_2"] = tmp_race_df["main_place_福島_2"].fillna(tmp_race_df["main_place_福島_1"])
tmp_race_df["main_place_阪神_2"] = tmp_race_df["main_place_阪神_2"].fillna(tmp_race_df["main_place_阪神_1"])
tmp_race_df["race_rank_2"] = tmp_race_df["race_rank_2"].fillna(tmp_race_df["race_rank_1"])
tmp_race_df["ground_type_ダ_2"] = tmp_race_df["ground_type_ダ_2"].fillna(tmp_race_df["ground_type_ダ_1"])
tmp_race_df["ground_type_芝_2"] = tmp_race_df["ground_type_芝_2"].fillna(tmp_race_df["ground_type_芝_1"])
tmp_race_df["ground_type_障_2"] = tmp_race_df["ground_type_障_2"].fillna(tmp_race_df["ground_type_障_1"])
tmp_race_df["horse_weight_dif_2"] = tmp_race_df["horse_weight_dif_2"].fillna(tmp_race_df["horse_weight_dif_1"])
tmp_race_df["same_jockey_2"] = tmp_race_df["same_jockey_2"].fillna(tmp_race_df["same_jockey_1"])

tmp_race_df["whole_horse_number_3"] = tmp_race_df["whole_horse_number_3"].fillna(tmp_race_df["whole_horse_number_2"])
tmp_race_df["odds_3"] = tmp_race_df["odds_3"].fillna(tmp_race_df["odds_2"])
tmp_race_df["order_3"] = tmp_race_df["order_3"].fillna(tmp_race_df["order_2"])
tmp_race_df["burden_weight_3"] = tmp_race_df["burden_weight_3"].fillna(tmp_race_df["burden_weight_2"])
tmp_race_df["race_distance_3"] = tmp_race_df["race_distance_3"].fillna(tmp_race_df["race_distance_2"])
tmp_race_df["ground_condition_3"] = tmp_race_df["ground_condition_3"].fillna(tmp_race_df["ground_condition_2"])
tmp_race_df["order_3"] = tmp_race_df["order_3"].fillna(tmp_race_df["order_2"])
tmp_race_df["goal_time_3"] = tmp_race_df["goal_time_3"].fillna(tmp_race_df["goal_time_2"])
#tmp_race_df["velocity_3"] = tmp_race_df["velocity_3"].fillna(tmp_race_df["velocity_2"])
tmp_race_df["half_order_3"] = tmp_race_df["half_order_3"].fillna(tmp_race_df["half_order_2"])
tmp_race_df["last_time_3"] = tmp_race_df["last_time_3"].fillna(tmp_race_df["last_time_2"])
tmp_race_df["horse_weight_3"] = tmp_race_df["horse_weight_3"].fillna(tmp_race_df["horse_weight_2"])
tmp_race_df["weather_circumstance_小雨_3"] = tmp_race_df["weather_circumstance_小雨_3"].fillna(tmp_race_df["weather_circumstance_小雨_2"])
tmp_race_df["weather_circumstance_小雪_3"] = tmp_race_df["weather_circumstance_小雪_3"].fillna(tmp_race_df["weather_circumstance_小雪_2"])
tmp_race_df["weather_circumstance_晴_3"] = tmp_race_df["weather_circumstance_晴_3"].fillna(tmp_race_df["weather_circumstance_晴_2"])
tmp_race_df["weather_circumstance_曇_3"] = tmp_race_df["weather_circumstance_曇_3"].fillna(tmp_race_df["weather_circumstance_曇_2"])
tmp_race_df["weather_circumstance_雨_3"] = tmp_race_df["weather_circumstance_雨_3"].fillna(tmp_race_df["weather_circumstance_雨_2"])
tmp_race_df["weather_circumstance_雪_3"] = tmp_race_df["weather_circumstance_雪_3"].fillna(tmp_race_df["weather_circumstance_雪_2"])
tmp_race_df["main_place_その他_3"] = tmp_race_df["main_place_その他_3"].fillna(tmp_race_df["main_place_その他_2"])
tmp_race_df["main_place_中京_3"] = tmp_race_df["main_place_中京_3"].fillna(tmp_race_df["main_place_中京_2"])
tmp_race_df["main_place_中山_3"] = tmp_race_df["main_place_中山_3"].fillna(tmp_race_df["main_place_中山_2"])
tmp_race_df["main_place_京都_3"] = tmp_race_df["main_place_京都_3"].fillna(tmp_race_df["main_place_京都_2"])
tmp_race_df["main_place_函館_3"] = tmp_race_df["main_place_函館_3"].fillna(tmp_race_df["main_place_函館_2"])
tmp_race_df["main_place_小倉_3"] = tmp_race_df["main_place_小倉_3"].fillna(tmp_race_df["main_place_小倉_2"])
tmp_race_df["main_place_新潟_3"] = tmp_race_df["main_place_新潟_3"].fillna(tmp_race_df["main_place_新潟_2"])
tmp_race_df["main_place_札幌_3"] = tmp_race_df["main_place_札幌_3"].fillna(tmp_race_df["main_place_札幌_2"])
tmp_race_df["main_place_東京_3"] = tmp_race_df["main_place_東京_3"].fillna(tmp_race_df["main_place_東京_2"])
tmp_race_df["main_place_福島_3"] = tmp_race_df["main_place_福島_3"].fillna(tmp_race_df["main_place_福島_2"])
tmp_race_df["main_place_阪神_3"] = tmp_race_df["main_place_阪神_3"].fillna(tmp_race_df["main_place_阪神_2"])
tmp_race_df["race_rank_3"] = tmp_race_df["race_rank_3"].fillna(tmp_race_df["race_rank_2"])
tmp_race_df["ground_type_ダ_3"] = tmp_race_df["ground_type_ダ_3"].fillna(tmp_race_df["ground_type_ダ_2"])
tmp_race_df["ground_type_芝_3"] = tmp_race_df["ground_type_芝_3"].fillna(tmp_race_df["ground_type_芝_2"])
tmp_race_df["ground_type_障_3"] = tmp_race_df["ground_type_障_3"].fillna(tmp_race_df["ground_type_障_2"])
tmp_race_df["horse_weight_dif_3"] = tmp_race_df["horse_weight_dif_3"].fillna(tmp_race_df["horse_weight_dif_2"])
tmp_race_df["same_jockey_3"] = tmp_race_df["same_jockey_3"].fillna(tmp_race_df["same_jockey_2"])



In [61]:
# print(tmp_race_df.shape)
# tmp_race_df.isnull().sum()

In [62]:
race_df = pd.merge(race_df, tmp_race_df, left_on = ["horse_id"], right_on = ["horse_id_1"], how = "left")

same_jockey

In [63]:
same_jockey = pd.DataFrame(columns = ["same_jockey"])
same_jockey["same_jockey"] = (race_df["jockey_id_1"] == race_df["jockey_id"]) * 1.0 

race_df = pd.concat([race_df,same_jockey], axis = 1)
race_df.drop(["jockey_id"], axis = 1,inplace = True)

In [64]:
race_df.drop(['horse_id_1','date_1'], axis = 1, inplace = True)

In [65]:
race_df["horse_id"] = race_df["horse_id"].drop_duplicates()
race_df = race_df.dropna(subset = {"horse_id"})

In [66]:
print(race_df.shape)
race_df.head(5)

(123, 154)


,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,age,burden_weight,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_中山,place_阪神,札幌,中京,新潟,函館,小倉,京都,東京,福島,sex_セ,sex_牝,sex_牡,f_grass_win_rate,f_dart_win_rate,f_win_rate,g_f_grass_win_rate,g_f_dart_win_rate,g_f_win_rate,...,main_place_中京_2,main_place_中山_2,main_place_京都_2,main_place_函館_2,main_place_小倉_2,main_place_新潟_2,main_place_札幌_2,main_place_東京_2,main_place_福島_2,main_place_阪神_2,race_rank_2,ground_type_ダ_2,ground_type_芝_2,ground_type_障_2,horse_weight_dif_2,same_jockey_2,whole_horse_number_3,odds_3,order_3,burden_weight_3,race_distance_3,ground_condition_3,goal_time_3,half_order_3,last_time_3,horse_weight_3,weather_circumstance_小雨_3,weather_circumstance_小雪_3,weather_circumstance_晴_3,weather_circumstance_曇_3,weather_circumstance_雨_3,weather_circumstance_雪_3,main_place_その他_3,main_place_中京_3,main_place_中山_3,main_place_京都_3,main_place_函館_3,main_place_小倉_3,main_place_新潟_3,main_place_札幌_3,main_place_東京_3,main_place_福島_3,main_place_阪神_3,race_rank_3,ground_type_ダ_3,ground_type_芝_3,ground_type_障_3,horse_weight_dif_3,same_jockey_3,same_jockey
0,202206050808&rf,8,冬至特別,1,2022-12-25,16,?,1,1,2019104518,3,54.0,?,?,?,3.6,?,0,1200,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.320000,0.0,0.320000,1.000000,0.0,1.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,0.0,10.0,1.7,2,54.0,1200.0,1.0,71.4,0.300000,37.1,448.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0
1,202206050808&rf,8,冬至特別,1,2022-12-25,16,?,1,2,2016110159,6,57.0,?,?,?,23.0,?,0,1200,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,16.0,2.2,1,57.0,1200.0,2.0,71.3,0.125000,36.9,510.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0
2,202206050808&rf,8,冬至特別,1,2022-12-25,16,?,2,3,2016101493,6,57.0,?,?,?,168.0,?,0,1200,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.312500,0.0,0.294118,0.000000,0.0,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,0.0,14.0,5.3,2,57.0,1000.0,1.0,58.4,0.178571,35.4,472.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-4.0,0.0,0.0
3,202206050808&rf,8,冬至特別,1,2022-12-25,16,?,2,4,2018101817,4,57.0,?,?,?,17.7,?,0,1200,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-2.0,0.0,16.0,13.6,2,57.0,1400.0,1.0,83.6,0.406250,36.1,454.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-4.0,0.0,0.0
4,202206050808&rf,8,冬至特別,1,2022-12-25,16,?,3,5,2017102984,5,55.0,?,?,?,90.8,?,0,1200,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.444444,0.0,0.444444,0.152174,0.0,0.152174,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0,0.0,16.0,20.9,1,55.0,1200.0,3.0,71.0,0.875000,36.0,420.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-6.0,0.0,0.0


In [67]:
race_df.to_csv("../data/csv/2022/Dec_4/12_25_data.csv", index = False)